In [1]:
!pip install catboost scikit-learn

In [2]:
!pip install pymystem3

In [3]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostClassifier, cv
from catboost.text_processing import Tokenizer
from pymystem3 import Mystem

In [4]:
df_answers = pd.read_csv('/Users/darabizina/reviews-summarization/reviews-summarization/notebooks/data/answers_202405192130.csv')
df_aspects = pd.read_csv('/Users/darabizina/reviews-summarization/reviews-summarization/notebooks/data/aspects_202405192134.csv')
df_reviews = pd.read_csv('/Users/darabizina/reviews-summarization/reviews-summarization/notebooks/data/reviews_202405192134.csv')


In [5]:
aspects_map = {
  df_aspects['id'][x]: df_aspects['aspect'][x]
  for x in range(df_aspects.shape[0])
}
reviews_map = {
  df_reviews['id'][x]: df_reviews['review_body'][x]
  for x in range(df_reviews.shape[0])
}

In [6]:
aspect_state = {
  1: 'хорошо',
  0: 'плохо',
  2: 'отсутствует',
  3: 'skip',
}

answers = pd.DataFrame(columns=['review', 'aspect', 'answer'])
for ind, row in df_answers.iterrows():
  review = reviews_map.get(row.review)
  aspect = aspects_map.get(row.aspect)
  answer = aspect_state[row.answer]
  answers.loc[ind] = [review, aspect, answer]

answers.head()

,review,aspect,answer
0,Какие только безумно-замечательные идеи не при...,Эмоциональное воздействие,плохо
1,"Этот фильм, можно сказать, путеводитель по стр...",Юмор,плохо
2,На фильм я пошел благодаря высокому рейтингу К...,Визуальные эффекты,отсутствует
3,Возможно ли прожить жизнь с улыбкой на лице? С...,Саундтрек/музыка,отсутствует
4,"Подмечу важное, фильмы 'о мальчике который выж...",Саундтрек/музыка,отсутствует


In [7]:
simple_tokenizer = Tokenizer(lowercasing=True, separator_type='BySense', token_types=['Word', 'Number', 'Punctuation'])

def tokenize_texts(texts):
    return [' '.join(simple_tokenizer.tokenize(text)) for text in texts]

In [8]:
mystem = Mystem()

def lemmatize_text(text):
    lemmas = mystem.lemmatize(text)
    return ''.join(lemmas).strip()

In [9]:
answers['review'] = answers['review'].apply(lambda x: ' '.join(simple_tokenizer.tokenize(x)))
answers['review'] = answers['review'].apply(lemmatize_text)
text_features = ['review']
cat_features = ['aspect']

In [10]:
def process(data):
  df = data.copy()
  df = df[df['aspect'] == 'Юмор']
  return df.drop(['answer'], axis=1), df['answer']

X, y = process(answers)

In [11]:
X.head()

,review,aspect
1,"этот фильм , можно сказать , путеводитель по с...",Юмор
5,покуда до сей пора не состояться цифро-релиз '...,Юмор
14,любить я советский кино вот любить и все нрави...,Юмор
19,"я смотреть этот фильм в детство , когда он вых...",Юмор
26,"давно смотреть этот фильм , но недавно вспомни...",Юмор


In [47]:
train_pool = Pool(
    data = X, 
    label = y, 
    cat_features=cat_features,
    text_features=text_features,
    feature_names=list(X)
)

model = CatBoostClassifier()

grid = {'learning_rate': [0.03, 0.1],
        }


# randomized_search_result = model.randomized_search(grid,
#                                                    train_pool,
#                                                    cv=5,                   n_iter=1)

print('AAAA')
model.fit(X, y, cat_features=['aspect'], text_features=['review'], early_stopping_rounds=10)

AAAA
Learning rate set to 0.069607
0:	learn: 1.0645608	total: 478ms	remaining: 7m 57s
1:	learn: 1.0362037	total: 780ms	remaining: 6m 29s
2:	learn: 1.0148447	total: 1.08s	remaining: 5m 59s
3:	learn: 0.9841280	total: 1.45s	remaining: 6m
4:	learn: 0.9604043	total: 1.81s	remaining: 6m 1s
5:	learn: 0.9366837	total: 2.2s	remaining: 6m 4s
6:	learn: 0.9203834	total: 2.52s	remaining: 5m 57s
7:	learn: 0.9065189	total: 2.81s	remaining: 5m 48s
8:	learn: 0.8913754	total: 3.11s	remaining: 5m 42s
9:	learn: 0.8780073	total: 3.42s	remaining: 5m 38s
10:	learn: 0.8636543	total: 3.71s	remaining: 5m 33s
11:	learn: 0.8422437	total: 4.01s	remaining: 5m 30s
12:	learn: 0.8277521	total: 4.31s	remaining: 5m 27s
13:	learn: 0.8154881	total: 4.61s	remaining: 5m 24s
14:	learn: 0.7980616	total: 4.91s	remaining: 5m 22s
15:	learn: 0.7814927	total: 5.2s	remaining: 5m 19s
16:	learn: 0.7702882	total: 5.49s	remaining: 5m 17s
17:	learn: 0.7597239	total: 5.83s	remaining: 5m 18s
18:	learn: 0.7506021	total: 6.18s	remaining: 5m

In [49]:
model.save_model('ml',
           format="cbm",
           export_parameters=None,
           pool=None)

In [51]:
print(__FILE__)

NameError: name '__FILE__' is not defined